# House Prices Preprosesing<a class="anchor" id="0-bullet"></a>

* [1. Loading libraries and data](#1-bullet)
  * [1.1 Categorical features](#1.1-bullet)
* [2. LightGBM Regresssor](#2-bullet)
  * [2.1 GridSearchCV](#2.1-bullet)
  * [2.2 Feature importances](#2.2-bullet)
  * [2.3 Result](#2.3-bullet)
* [3. CatBoostRegressor](#3-bullet)
* [4. Final Result](#4-bullet)

## 1. Loading libraries and data<a class="anchor" id="1-bullet"></a>
[back to the table of contents](#0-bullet)

In [32]:
!pip install catboost

     |████████████████████████████████| 67.4 MB 27 kB/s 


In [33]:
#for displaying all pins - not only last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

#data
import pandas as pd
import numpy as np

#for learning baseline-model
from sklearn.model_selection import GridSearchCV

import lightgbm
import catboost
from catboost import CatBoostRegressor

In [8]:
cd /content/drive/MyDrive/House Prices

/content/drive/MyDrive/House Prices


In [9]:
df_train_x = pd.read_csv('/data/df_train_x')
df_test = pd.read_csv('/data/df_test')
train_global = pd.read_csv('/data/train.csv')
df_train_y = train_global['SalePrice']

### 1.1 Categorical features <a class="anchor" id="1.1-bullet"></a>

In [12]:
all_cat_features = [
    'MSZoning', 'Street', 'Alley', 'LotShape', 
    'LandContour', 'Utilities', 'LotConfig', 
    'LandSlope', 'Neighborhood', 'Condition1',
    'Condition2', 'BldgType', 'HouseStyle',
    'RoofStyle', 'RoofMatl', 'Exterior1st',
    'Exterior2nd', 'MasVnrType', 'ExterQual',
    'ExterCond', 'Foundation', 'BsmtQual',
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
    'BsmtFinType2', 'Heating', 'HeatingQC',
    'CentralAir', 'Electrical', 'KitchenQual',
    'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',
    'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC',
    'Fence', 'MiscFeature', 'SaleType', 'SaleCondition', 
    'MSSubClass', 'OverallQual', 'OverallCond', 'BsmtFullBath',
    'BsmtHalfBath', 'FullBath', 'HalfBath','BedroomAbvGr',
    'KitchenAbvGr','TotRmsAbvGrd','Fireplaces', 'GarageAge', 'RemodAge', 'Cat_RemodAge',
    'GarageCars', 'MoSold', 'YrSold', 'GarageYrBlt', 'YearBuilt', 'YearRemodAdd', 'YrBltAndRemod', 'YearsSinceRemodel', 'HouseAge',   'Cat_HouseAge']

In [36]:
#change all categorical numeric values to strings - it's nessesary for trainig CatboostReg
for x in all_cat_features:
    df_train_x[x] = df_train_x[x].astype(str)
  
for x in all_cat_features:
    df_test[x] = df_test[x].astype(str)

In [55]:
#change all categorical values to category
for c in all_cat_features:
    df_train_x[c] = df_train_x[c].astype('category')

for c in all_cat_features:
    df_test[c] = df_test[c].astype('category')

## 2. LightGBM Regresssor <a class="anchor" id="2-bullet"></a>
[back to the table of contents](#0-bullet)

In [39]:
lgbm = lightgbm.LGBMRegressor(learning_rate=0.07,
                              categorical_feature = all_cat_features) 

### 2.1 GridSearchCV <a class="anchor" id="2.1-bullet"></a>

In [40]:
param_grid = {
    'n_estimators': [600, 700, 800, 7000 ],
    'max_depth': np.arange(4, 10, 30) #[i for i in range(10, 30)]
}
cv = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error', 
    cv=5, 
    n_jobs=-1
)

In [41]:
cv.fit(df_train_x, df_train_y)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:741: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  .format(key))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt',
                                     categorical_feature=['MSZoning', 'Street',
                                                          'Alley', 'LotShape',
                                                          'LandContour',
                                                          'Utilities',
                                                          'LotConfig',
                                                          'LandSlope',
                                                          'Neighborhood',
                                                          'Condition1',
                                                          'Condition2',
                                                          'BldgType',
                                                          'HouseStyle',
                                                          'RoofStyle',
                             

In [42]:
cv.best_params_
lgbm_best = lightgbm.LGBMRegressor(**cv.best_params_,learning_rate=0.07,
                              categorical_feature = all_cat_features)

{'max_depth': 4, 'n_estimators': 600}

In [43]:
lgbm_best.fit(df_train_x, df_train_y)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:741: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  .format(key))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


LGBMRegressor(boosting_type='gbdt',
              categorical_feature=['MSZoning', 'Street', 'Alley', 'LotShape',
                                   'LandContour', 'Utilities', 'LotConfig',
                                   'LandSlope', 'Neighborhood', 'Condition1',
                                   'Condition2', 'BldgType', 'HouseStyle',
                                   'RoofStyle', 'RoofMatl', 'Exterior1st',
                                   'Exterior2nd', 'MasVnrType', 'ExterQual',
                                   'ExterCond', 'Foundation', 'BsmtQual',
                                   'BsmtCond', 'BsmtExposure', 'B...
                                   'CentralAir', 'Electrical', ...],
              class_weight=None, colsample_bytree=1.0, importance_type='split',
              learning_rate=0.07, max_depth=4, min_child_samples=20,
              min_child_weight=0.001, min_split_gain=0.0, n_estimators=600,
              n_jobs=-1, num_leaves=31, objective=None, random_state

### 2.2 Feature importances <a class="anchor" id="2.2-bullet"></a>

In [44]:
best_features = pd.DataFrame(lgbm_best.feature_importances_)

In [45]:
idx = lgbm_best.feature_importances_

best_features = pd.DataFrame(idx, index = df_train_x.columns)
best_features.rename(columns={0:'degree_of_influence'}, inplace =True)
best_features = best_features.query('degree_of_influence > 0').sort_values(by='degree_of_influence', ascending=False)
#best_features = best_features.query('degree_of_influence >= 91').sort_values(by='degree_of_influence', ascending=False)
list_best_features = best_features.index
list_best_features

Index(['LotArea', 'TotalSF', 'Neighborhood', 'TotalBsmtSF', 'BsmtFinSF1',
       'GrLivArea', 'BsmtUnfSF', 'LotFrontage', 'GarageArea',
       'YearsSinceRemodel', '1stFlrSF', 'MoSold', 'MasVnrArea', 'OpenPorchSF',
       'GarageYrBlt', '2ndFlrSF', 'WoodDeckSF', 'YearBuilt', 'YrBltAndRemod',
       'YearRemodAdd', 'OverallQual', 'BsmtExposure', 'OverallCond',
       'Fireplaces', 'GarageFinish', 'Exterior1st', 'TotRmsAbvGrd', 'FullBath',
       'KitchenQual', 'LandContour', 'LotShape', 'MSSubClass', 'BsmtFullBath',
       'Cat_HouseAge', 'HeatingQC', 'BsmtFinType1', 'Condition1',
       'Cat_RemodAge', 'ExterQual', 'SaleCondition', 'HalfBath', 'CentralAir',
       'BsmtCond', 'YrSold', 'ExterCond', 'Exterior2nd', 'BedroomAbvGr',
       'Functional', 'Electrical', 'GarageType', 'Foundation', 'MasVnrType',
       'BsmtHalfBath', 'LotConfig', 'FireplaceQu', 'BsmtQual', 'GarageCars',
       'Alley', 'LandSlope', 'SaleType', 'Fence', 'MSZoning', 'RoofMatl',
       'PavedDrive', 'KitchenAbvG

In [46]:
list_best_all_cat_features = [x for x in list_best_features if x in all_cat_features]

In [47]:
lgbm_best_feature = lightgbm.LGBMRegressor(**cv.best_params_,learning_rate=0.07,
                              categorical_feature = list_best_all_cat_features) 

In [48]:
lgbm_best_feature.fit(df_train_x[list_best_features], np.log(df_train_y))

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:741: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  .format(key))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


LGBMRegressor(boosting_type='gbdt',
              categorical_feature=['Neighborhood', 'YearsSinceRemodel',
                                   'MoSold', 'GarageYrBlt', 'YearBuilt',
                                   'YrBltAndRemod', 'YearRemodAdd',
                                   'OverallQual', 'BsmtExposure', 'OverallCond',
                                   'Fireplaces', 'GarageFinish', 'Exterior1st',
                                   'TotRmsAbvGrd', 'FullBath', 'KitchenQual',
                                   'LandContour', 'LotShape', 'MSSubClass',
                                   'BsmtFullBath', 'Cat_HouseAge',...
                                   'CentralAir', 'BsmtCond', ...],
              class_weight=None, colsample_bytree=1.0, importance_type='split',
              learning_rate=0.07, max_depth=4, min_child_samples=20,
              min_child_weight=0.001, min_split_gain=0.0, n_estimators=600,
              n_jobs=-1, num_leaves=31, objective=None, random_state=None,

### 2.3 Result <a class="anchor" id="2.3-bullet"></a>

In [49]:
y_pred_2 = np.exp(lgbm_best_feature.predict(df_test[list_best_features]))

In [50]:
submission = pd.read_csv('sample_submission.csv')
submission['SalePrice'] = y_pred_2
submission.to_csv('submission_2', index=False)

## 3. CatBoost Regresssor <a class="anchor" id="3-bullet"></a>
[back to the table of contents](#0-bullet)

In [51]:
catbst = catboost.CatBoostRegressor(depth = 4, iterations = 600, learning_rate=0.07,
                              cat_features = list_best_all_cat_features)


In [52]:
catbst.fit(df_train_x[list_best_features], np.log(df_train_y))

0:	learn: 0.3813701	total: 69.6ms	remaining: 41.7s
1:	learn: 0.3649855	total: 91.7ms	remaining: 27.4s
2:	learn: 0.3504213	total: 110ms	remaining: 21.9s
3:	learn: 0.3367435	total: 128ms	remaining: 19.1s
4:	learn: 0.3236043	total: 142ms	remaining: 16.9s
5:	learn: 0.3116488	total: 161ms	remaining: 16s
6:	learn: 0.2993609	total: 179ms	remaining: 15.2s
7:	learn: 0.2882956	total: 197ms	remaining: 14.6s
8:	learn: 0.2789472	total: 216ms	remaining: 14.2s
9:	learn: 0.2694083	total: 233ms	remaining: 13.8s
10:	learn: 0.2605752	total: 250ms	remaining: 13.4s
11:	learn: 0.2518589	total: 271ms	remaining: 13.3s
12:	learn: 0.2441913	total: 287ms	remaining: 13s
13:	learn: 0.2372939	total: 304ms	remaining: 12.7s
14:	learn: 0.2305619	total: 322ms	remaining: 12.5s
15:	learn: 0.2246694	total: 338ms	remaining: 12.3s
16:	learn: 0.2190849	total: 358ms	remaining: 12.3s
17:	learn: 0.2139758	total: 374ms	remaining: 12.1s
18:	learn: 0.2083928	total: 390ms	remaining: 11.9s
19:	learn: 0.2030877	total: 406ms	remaining

## 4. Final result <a class="anchor" id="4-bullet"></a>
[back to the table of contents](#0-bullet)

In [56]:
y_pred_3 = np.exp(catbst.predict(df_test[list_best_features]))

In [57]:
submission = pd.read_csv('sample_submission.csv')
submission['SalePrice'] = y_pred_3
submission.to_csv('submission_3', index=False)